# Ordering


## Warning: This notebook will place live orders

Use a papertrading account when running this notebook.


In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 4002, clientId=23)

ERROR:ib_insync.wrapper:Error 321, reqId 12: Error validating request:-'aS' : cause - ALL account is not supported


<IB connected to 127.0.0.1:4002 clientId=23>

Create a contract and a market order:

In [2]:
contract = Stock('AAPL', 'SMART', 'USD')
ib.qualifyContracts(contract)

order = MarketOrder('BUY', 100)

placeOrder will place the order order and return a ``Trade`` object right away (non-blocking):

In [3]:
trade = ib.placeOrder(contract, order)

trade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=MarketOrder(orderId=15, action='BUY', totalQuantity=100), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 52, 14, 113511, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``trade`` contains the order and everything related to it, such as order status, fills and a log.
It will be live updated with every status change or fill of the order.

In [4]:
ib.sleep(1)
trade.log

[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 52, 14, 113511, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 52, 14, 627143, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='')]

``trade`` will also available from ``ib.trades()``:

In [5]:
assert trade in ib.trades()

Likewise for ``order``:

In [6]:
assert order in ib.orders()

Now let's create a limit order with an unrealistic limit:

In [7]:
limitOrder = LimitOrder('BUY', 100, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=16, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PendingSubmit'), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 502037, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='')])

``status`` will change from "PendingSubmit" to "Submitted":

In [8]:
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

AssertionError: 

In [9]:
assert limitTrade in ib.openTrades()

Let's modify the limit price and resubmit:

In [10]:
limitOrder.lmtPrice = 0.10

ib.placeOrder(contract, limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=16, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='PreSubmitted', remaining=100.0, permId=2113698761, clientId=23, whyHeld='locate', mktCapPrice=None), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 502037, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 825797, tzinfo=datetime.timezone.utc), status='PreSubmitted', message=''), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 54, 26, 409070, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Modify')])

And now cancel it:

In [11]:
ib.cancelOrder(limitOrder)

Trade(contract=Stock(conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), order=LimitOrder(orderId=16, action='BUY', totalQuantity=100, lmtPrice=0.1), orderStatus=OrderStatus(status='PreSubmitted', remaining=100.0, permId=2113698761, clientId=23, whyHeld='locate', mktCapPrice=None), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 502037, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 825797, tzinfo=datetime.timezone.utc), status='PreSubmitted', message=''), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 54, 26, 409070, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Modify'), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 54, 33, 554896, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

In [12]:
limitTrade.log

[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 502037, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 53, 19, 825797, tzinfo=datetime.timezone.utc), status='PreSubmitted', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 54, 26, 409070, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='Modify'),
 TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 54, 33, 554896, tzinfo=datetime.timezone.utc), status='PendingCancel', message=''),
 TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 54, 33, 907979, tzinfo=datetime.timezone.utc), status='Cancelled', message='')]

Some note of warning: reqContractDetails can happily report that a contract is unique,
while placeOrder() will complain that the contract is ambiguous. An example is INTC:

In [13]:
intc = Stock('INTC', 'SMART', 'USD')

assert len(ib.reqContractDetails(intc)) == 1

intcOrder = LimitOrder('BUY', 100, 0.05)
trade = ib.placeOrder(intc, intcOrder)
ib.sleep(1);

ERROR:ib_insync.wrapper:Error 200, reqId 18: The contract description specified for INTC is ambiguous.


In [14]:
assert trade.orderStatus.status == 'Cancelled'

So it is always a good
idea to a qualify a contract before trading it.

In [15]:
ib.qualifyContracts(intc)

intcOrder = LimitOrder('BUY', 100, 0.05)
trade = ib.placeOrder(intc, intcOrder)

In [16]:
ib.cancelOrder(intcOrder)

Trade(contract=Stock(conId=270639, symbol='INTC', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='INTC', tradingClass='NMS'), order=LimitOrder(orderId=20, action='BUY', totalQuantity=100, lmtPrice=0.05), orderStatus=OrderStatus(status='PreSubmitted', remaining=100.0, permId=2113698762, clientId=23, whyHeld='locate', mktCapPrice=None), fills=[], log=[TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 55, 30, 988670, tzinfo=datetime.timezone.utc), status='PendingSubmit', message=''), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 55, 31, 288631, tzinfo=datetime.timezone.utc), status='PreSubmitted', message=''), TradeLogEntry(time=datetime.datetime(2018, 6, 22, 5, 56, 0, 675344, tzinfo=datetime.timezone.utc), status='PendingCancel', message='')])

placeOrder is not blocking and will not wait on what happens with the order.
To make the order placement blocking, that is to wait until the order is either
filled or canceled, consider the following:

In [17]:
%%time
order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

KeyboardInterrupt: 

What are our positions?

In [18]:
ib.positions()

[Position(account='DU774248', contract=Contract(secType='STK', conId=10098, symbol='BAC', exchange='NYSE', currency='USD', localSymbol='BAC', tradingClass='BAC'), position=300.0, avgCost=23.835),
 Position(account='DU774248', contract=Contract(secType='CASH', conId=12087802, symbol='CHF', currency='USD', localSymbol='CHF.USD', tradingClass='CHF.USD'), position=30000.0, avgCost=1.0082),
 Position(account='DU774248', contract=Contract(secType='CASH', conId=14433401, symbol='AUD', currency='USD', localSymbol='AUD.USD', tradingClass='AUD.USD'), position=150000.0, avgCost=0.7881105),
 Position(account='DU774248', contract=Contract(secType='STK', conId=4725951, symbol='AEO', exchange='NYSE', currency='USD', localSymbol='AEO', tradingClass='AEO'), position=500.0, avgCost=10.605),
 Position(account='DU774248', contract=Contract(secType='CASH', conId=12087792, symbol='EUR', currency='USD', localSymbol='EUR.USD', tradingClass='EUR.USD'), position=8600000.0, avgCost=1.18190765),
 Position(account

What's the total of commissions paid today?

In [19]:
sum(fill.commissionReport.commission for fill in ib.fills())

0

whatIfOrder can be used to see the commission and the margin impact of an order without actually sending the order:

In [20]:
order = MarketOrder('SELL', 200)
ib.whatIfOrder(contract, order)

OrderState(status='PreSubmitted', initMargin='450357.7', maintMargin='411864.39', equityWithLoan='1402339.82', minCommission=1.265372, maxCommission=1.505372, commissionCurrency='USD')

In [21]:
ib.disconnect()

***